# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 17
REFERENCE_PANEL = "1000G"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: 1000G"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/gene_corrs-chr17.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite/gene_corrs-chr17.pkl"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  # eigenvalues <- eigen(data)$values
  # nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  cholStatus <- try(u <- chol(data), silent = FALSE)
  cholError <- ifelse(class(cholStatus) == "try-error", TRUE, FALSE)

  if (cholError) {
    return(FALSE)
  } else {
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 411 411

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000141252,ENSG00000179409,ENSG00000177370,ENSG00000159842,ENSG00000108953,ENSG00000167193,ENSG00000132376,ENSG00000174238,ENSG00000167703,ENSG00000074660
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000141252,1.00000000,0.35780789,0.010011849,0.01377192,0.00444020,0.004143520,0.004822640,0.003588880,0.007753406,0.002159075
ENSG00000179409,0.35780789,1.00000000,0.014970151,0.01770324,0.01251407,0.009100100,0.008721908,0.005277922,0.016283745,0.006706836
ENSG00000177370,0.01001185,0.01497015,1.000000000,0.02570040,0.00899150,0.007758048,0.004638207,0.008483488,0.015865313,0.006262646
ENSG00000159842,0.01377192,0.01770324,0.025700403,1.00000000,0.02891454,0.016799926,0.015119181,0.012697359,0.022391914,0.013776858
ENSG00000108953,0.00444020,0.01251407,0.008991500,0.02891454,1.00000000,0.138716139,0.027416178,0.021850641,0.019317545,0.005181194
ENSG00000167193,0.00414352,0.00910010,0.007758048,0.01679993,0.13871614,1.000000000,0.037590146,0.024752790,0.107401072,0.035616018


# Check positive definiteness

In [17]:
orig_matrix_is_positive_definite <- is_positive_definite(gene_corrs)

In [18]:
if (orig_matrix_is_positive_definite) {
  IRdisplay::display("Matrix is already positive definite!")
  file.copy(INPUT_FILE, OUTPUT_FILE)
} else {
  IRdisplay::display("We need to correct the data and make the matrix positive definite")

  eigenvalues <- eigen(gene_corrs)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
  IRdisplay::display(length(nonpositive_eigenvalues))
  IRdisplay::display(nonpositive_eigenvalues)

  # Make matrix positive definite
  gene_corrs_corrected <- CorrectCM(gene_corrs, 1e-10)
  dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)
  dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)
  gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

  # check if new matrix is really positive definite
  matrix_is_positive_definite <- is_positive_definite(gene_corrs_corrected)
  if (matrix_is_positive_definite) {
    IRdisplay::display("It worked!")
    IRdisplay::display(head(gene_corrs[1:10, 1:10]))
    IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))
    py_save_object(gene_corrs_corrected, OUTPUT_FILE)
  } else {
    eigenvalues <- eigen(gene_corrs_corrected)$values
    nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
    IRdisplay::display(length(nonpositive_eigenvalues))
    IRdisplay::display(nonpositive_eigenvalues)

    stop("Method failed to adjust matrix")
  }
}

[1] "Matrix is already positive definite!"

[1] TRUE

Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).